# iFest UNPAD Yaudahlah: Topic Classifier

Kurniadi Ahmad Wijaya<sup>1</sup><br>
Kaenova Mahendra Auditama<sup>2</sup><br>
Fendi Irfan Amorokhman<sup>3</sup><br>
Ananda Affan Fattahila<sup>4</sup><br>
<sup>1</sup><a href="mailto:kurniadiwijaya@student.telkomuniversity.ac.id">kurniadiwijaya@student.telkomuniversity.ac.id</a><br>
<sup>2</sup><a href="mailto:kaenova@student.telkomuniversity.ac.id">kaenova@student.telkomuniversity.ac.id</a><br>
<sup>3</sup><a href="mailto:fendiirfan@student.telkomuniversity.ac.id">fendiirfan@student.telkomuniversity.ac.id</a><br>
<sup>4</sup><a href="mailto:affanfattahila@student.telkomuniversity.ac.id">affanfattahila@student.telkomuniversity.ac.id</a><br>
Informatics Engineering, Telkom University, Indonesia<br>
2021

In [ ]:
# Optional
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

class TopicClassifier:
  topics = {
    "pengaksesan" : ['login', 'register', 'kode', 'otp', 'pin'],
    "transaksi"   : [
                      'transfer', 'pembelian', 'pembayaran', 'saldo', 'pulsa', 'topup',
                      'top', 'up', 'beli', 'listrik', 'transaksi', 'biaya', 'potongan', 'uang', 
                    ],
    "akun"        : ['upgrade', 'verifikasi', 'ktp', 'premium', 'akun'],
    "cs"          : ['email', 'wa', 'whatsapp', 'telpon', 'cs', 'telfon', 'respon', 'customer service'],
    "performa"    : ['lambat', 'lag', 'cepat', 'performa']
  }

  @classmethod
  def OneHotEncodingDataFrameClassifier(cls, dataFrame, text_column:str):
    '''
    Akan mengembalikan df dengan kolom text serta one hot encoding dari topik-topik yang didefinisikan.
    '''
    # Copying Dataframe
    df = dataFrame.copy()

    # Initializing final Data
    final_dict = {
        'text' : []
    }
    for topic in cls.topics.keys():
      final_dict[topic] = np.zeros(shape= (len(df),))
    
    print("Tahap 1")
    for i in tqdm(range(len(df))):
      text = df.loc[i][text_column]
      final_dict['text'].append(text)
      classification = cls.__classify__(text)
      for topic in classification:
        final_dict[topic][i] = 1

    final_df = pd.DataFrame(final_dict)
    
    # Deleting text with all zeros on topics
    print("Tahap 2")
    for i in tqdm(range(len(final_df))):
      jum = 0
      for topic in cls.topics.keys():
        jum += final_df.loc[i][topic]

      if jum == 0:
        final_df = final_df.drop(i)
    
    final_df = final_df.reset_index(drop=True)

    return final_df

  @classmethod
  def CreateDataFramePerTopic(cls, dataFrame, destinationPath, namaOutputFile, text_column):
    '''
    Akan membuat csv dari setiap topic yang sudah didefinisikan dan dari dataFrame yang dimasukkan, serta text berpacu kepada parameter (text_column)
    '''
    # Initialization
    df = dataFrame.copy()
    final_df_dict = {}
    for topic in cls.topics.keys():
      final_df_dict[topic] = pd.DataFrame(columns=df.columns)
    if destinationPath[len(destinationPath)-1] != "/":
      destinationPath += "/"
    
    # Classifying
    for i in tqdm(range(len(df))):
      text = df.loc[i][text_column]
      classification = cls.__classify__(text)
      for topic in classification:
        final_df_dict[topic] = final_df_dict[topic].append(df.iloc[i],ignore_index = True)

    # Output
    for topic in list(final_df_dict.keys()):
      final_df_dict[topic].to_csv("{}.csv".format((destinationPath + namaOutputFile + "_{}".format(topic))), index=False)

  @classmethod
  def CreateDictionaryPerTopic(cls, dataFrame, text_column):
    '''
    Akan mereturn dictionary yang berisikan setiap topic yang sudah didefinisikan dan dari dataFrame yang dimasukkan, serta text berpacu kepada parameter (text_column)
    '''
    # Initialization
    df = dataFrame.copy()
    final_df_dict = {}
    for topic in cls.topics.keys():
      final_df_dict[topic] = pd.DataFrame(columns=df.columns)
    
    # Classifying
    for i in tqdm(range(len(df))):
      text = df.loc[i][text_column]
      classification = cls.__classify__(text)
      for topic in classification:
        final_df_dict[topic] = final_df_dict[topic].append(df.iloc[i],ignore_index = True)

    # Output
    return final_df_dict


  @classmethod
  def TextClassifying(cls, text):
    return cls.__classify__(text)

  @classmethod
  def __classify__(cls, text):
    final_class = []
    text_split = text.split(' ')
    for i in range(len(text_split)):
      kata = text_split[i]
      for topic in cls.topics.keys():
        if kata in cls.topics[topic]:
          final_class.append(topic)
    
    return list(set(final_class))

In [ ]:
#@title One Hot Encoding Multiple CSV from a Folder (Jangan Gunakan Yang Ini)

import os
directory = '/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/data/processedV4' #@param {type:"string"}
out_directory = '/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/data/autoLabelV2' #@param {type:"string"}

if directory[len(directory)-1] != '/':
  directory = directory + '/'

if out_directory[len(out_directory)-1] != '/':
  out_directory = out_directory + '/'

if not os.path.exists(out_directory):
  os.mkdir(out_directory)
  print("Directory Created at {}".format(out_directory))

files = os.listdir(directory)

for file in files:
    if file[len(file)-4:] == '.csv':
      print(file)
      df = pd.read_csv(directory+file)
      df_temp = TopicClassifier.OneHotEncodingDataFrameClassifier(df, 'processed')
      df_temp.to_csv(out_directory+file, index=False)

dana.csv
Tahap 1


100%|██████████| 3164/3164 [00:00<00:00, 7039.20it/s]


Tahap 2


100%|██████████| 3164/3164 [00:02<00:00, 1084.55it/s]


ovo.csv
Tahap 1


...

100%|██████████| 844/844 [00:00<00:00, 1017.36it/s]


link.csv
Tahap 1


100%|██████████| 1580/1580 [00:00<00:00, 7269.53it/s]


Tahap 2


100%|██████████| 1580/1580 [00:01<00:00, 1113.08it/s]


In [ ]:
#@title Create Multiple CSV Topic from a Folder (Gunakan yang Ini)

import os
directory = '/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/data/predictV1' #@param {type:"string"}
out_directory = '/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/data/classifiedV1' #@param {type:"string"}

if directory[len(directory)-1] != '/':
  directory = directory + '/'

if not os.path.exists(directory):
  raise "Input Directory is not exist"

if out_directory[len(out_directory)-1] != '/':
  out_directory = out_directory + '/'

if not os.path.exists(out_directory):
  os.mkdir(out_directory)
  print("Directory Created at {}".format(out_directory))

files = os.listdir(directory)

for file in files:
    if file[len(file)-4:] == '.csv':
      print(file)
      nama_file = file[:len(file)-4]
      df = pd.read_csv(directory+file)
      TopicClassifier.CreateDataFramePerTopic(df, out_directory, nama_file, 'raw')

link.csv


100%|██████████| 1580/1580 [00:05<00:00, 277.99it/s]


sakuku.csv


100%|██████████| 844/844 [00:02<00:00, 324.34it/s]


dana.csv


100%|██████████| 3164/3164 [00:14<00:00, 223.43it/s]


ovo.csv


100%|██████████| 2009/2009 [00:07<00:00, 256.06it/s]


In [ ]:
final_dict = TopicClassifier.CreateDictionaryPerTopic(df, 'raw')

100%|██████████| 2009/2009 [00:07<00:00, 254.78it/s]


In [ ]:
input = "aduh jadi min tiba login sandi salah benar loh coba bekukan jam maksudnya ya perbaiki akun mengirim uang ribu"#@param {type:"string"}
TopicClassifier.TextClassifying(input)

['transaksi', 'pengaksesan', 'akun']